<div >
    <img src = "../Banner.jpg" />
</div>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ignaciomsarmiento/IAMD/blob/main/Rec_Usuarios/Rec_Systems.ipynb)


# Sistemas de Recomendación

## ¿Qué son los sistemas de recomendaciones?


Las preferencias de los individuos suelen seguir patrones que los sistemas de recomendación pueden aprovechar, por ejemplo,

- Si te interesó:  <div style="max-width:200px">
<img src = "figs/iron_man.jpg" />
</div>



- También te puede interesar: <div style="max-width:200px">
<img src = "figs/thor.jpg" />
</div> 




<div  style="max-width: 80%;">
<img src = "figs/item_jeans.png"/>
</div>



<div  style="max-width: 80%;">
<img src = "figs/usuario_libros.png"/>
</div>

- Los sistemas de recomendación entonces encuentran patrones que son utilizados para predecir qué otros productos podrían gustarnos y generar sugerencias, de forma tal que  los usuarios encuentren contenido atractivo en un gran corpus. 

- Estos sistemas son muy exitosos, por ejemplo según un estudio del 2013 de [McKinsey](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers), el 35% de los artículos comprados en Amazon surgen de estos sistemas de recomendación, y por lo tanto vale la pena estudiarlos cuidadosamente.





<div style="max-width:500px">
<img src = "figs/BING.png" />
</div>


## Introducción: el problema de predicción 

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con $m$ usuarios y $n$ ítems. En esta matriz cada fila representa un usuario y cada columna un ítem. El valor de la celda denota el rating que le dió cada usuario al ítem. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al ítem $j$. Un ejemplo de esta matriz sería la siguiente:

<div  style="max-width: 40%;">
<img src = "figs/fig1.png"/>
</div>

Esta matriz tiene 7 usuarios ($m=7$) y 6 ítems ($n=6$). El usuario 1 le otorgó al ítem 1 un rating de 4, por lo tanto $r_{11}=4$. Sin embargo, no todos los usuarios calificaron/utilizaron todos los ítems y por lo tanto no todos tienen ranking. En el caso del usuario 1 este no utilizó los ítems 3, 4, y 6; por lo tanto aparecen con un signo de interrogación. Estos datos están faltando en la matriz.


Consideremos otro ejemplo más concreto. Imaginemos que somos una compañía como Netflix y tenemos un repositorio de 20.000 películas y 5.000 usuarios. Tenemos además un sistema que registra la calificación que cada usuario le otorga a una película en particular. Es decir, tenemos una  matriz de tamaño 5,000 × 20,000. Sin embargo, es muy probable que los usuarios sólo habrán visto sólo una fracción de las películas (difícilmente todos vieron 20.000 películas!). Por lo tanto, la matriz será poco densa, la mayoría de las entradas en la matriz estarán vacías.


El problema de predicción, tiene por objetivo predecir estos valores faltantes utilizando toda la información que tenemos a disposición: las calificaciones registradas, datos sobre películas, datos sobre usuarios, etc. Si el sistema es capaz de predecir con precisión los valores que faltan, podrá dar excelentes recomendaciones. Por ejemplo, si el usuario $i$ no ha utilizado el ítem $j$, pero nuestro sistema predice una calificación muy alta, es entonces muy probable que le guste $j$ si lo descubre a través del sistema de recomendación.

## Filtrado Colaborativo Basado en Usuarios.

El filtrado colaborativo aprovecha el poder de la colaboración para generar recomendaciones. 

 <div style="max-width:400px">
<img src = "figs/Colab.png" />
</div> 



### Ejemplo

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con 5 usuarios y 5 productos, en este caso restaurantes. 

El valor de la celda denota el rating que le dió cada usuario al restaurante. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al restaurante $j$. 
 

In [1]:
#cargamos librerias
import pandas as pd
import numpy as np

# Cargamos y visualizamos  los datos
ratings = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/Ratings.csv')
ratings.head()

,restaurant_id,user_id,rating
0,1,1,5.0
1,1,2,4.0
2,1,3,3.0
3,1,4,NaN
4,1,5,NaN


In [2]:
# pivotamos la tabla para tener una matriz de usuarios y restaurantes
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='restaurant_id')
r_matrix.head()

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,NaN,NaN
2,4.0,3.0,4.0,NaN,NaN
3,3.0,2.0,3.0,NaN,NaN
4,NaN,NaN,NaN,4.0,5.0
5,NaN,NaN,NaN,4.0,5.0


In [3]:
# creamos una matriz de usuarios y restaurantes donde remplazamos los valores nulos por 0
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,0.0,0.0
2,4.0,3.0,4.0,0.0,0.0
3,3.0,2.0,3.0,0.0,0.0
4,0.0,0.0,0.0,4.0,5.0
5,0.0,0.0,0.0,4.0,5.0


### Filtrado colaborativo sencillo (simple): medias, y medias ponderadas.


#### Medias 

- Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario 

In [4]:
 puntuac_promedio = ratings.groupby('restaurant_id').mean()
 pd.DataFrame(puntuac_promedio.iloc[:,1])

,rating
restaurant_id,
1,4.0
2,3.0
3,4.0
4,4.0
5,5.0


#### Medias ponderadas

- Podemos hacer algo más sofisticado usando recomendaciones de usuarios similares?


- Podemos hacer una media ponderada


$$
r_{ur}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'r}}{\sum_{u',u'\neq u}sim(u,u')}
$$



- Es decir la predicción del rating del usuario $u$ para el restaurante $r$, $r_{ur}$, es la suma ponderada de los ratings de los otros usuarios ($u'$) a este restaurante, 


- Ponderado por cuán similares son los usuarios $u'$ a $u$. 


- Como medimos similitud?

    - Existen múltiples medidas de distancia que se utilizan para medir la similitud. 

    - La distancia de coseno, que suele ser la más utilizada en los sistemas de recomendación.

    - Matemáticamente

$$
coseno(x,y)=\frac{x.y'}{|x||y|}
$$


Es decir, es el cociente del producto punto, dividido por las normas de los vectores.

<center>
<img src = "figs/dist_cos.png" alt = "coseno" style = "width: 300px;"/>
</center>


- Si el ángulo es 0  de grados, entonces los vectores se solapan, y el coseno es igual a 1 
- Si el ángulo es 90 de grados, los vectores forman un angulo recto, y el coseno es igual a 0.
- Si el ángulo es 180 de grados, los vectores estan en sentido opuesto, y  el coseno es igual a -1.



In [5]:
import math

math.cos(math.radians(0))

1.0

**¿Qué medida de similitud elegir?** 

Notemos que, si dos vectores, $x$ e $y$, tienen media cero, el coseno del angulo entre ambos coincidirá con el coeficiente de correlación de Pearson.

La distancia del coseno suele ser una medida de similitud ampliamente utilizada en los sistemas de recomendación por varias razones:

   - *Independencia de magnitud*: La distancia del coseno mide la similitud entre dos vectores basándose en el ángulo entre ellos, en lugar de depender de las magnitudes de los vectores. Esto es útil en los sistemas de recomendación, donde las magnitudes de las características (por ejemplo, la calificación de un usuario para un artículo) pueden variar significativamente entre diferentes usuarios o elementos. Al ser independiente de la magnitud, el coseno permite capturar patrones de similitud más robustos.

   - *Espacios de alta dimensión*: En los sistemas de recomendación, las características que representan usuarios o elementos a menudo se representan como vectores en espacios de alta dimensión. En estos espacios, la noción de distancia euclidiana puede volverse menos significativa debido al fenómeno conocido como "maldición de la dimensionalidad". La distancia del coseno puede ser más adecuada en estos casos porque se centra en la orientación relativa de los vectores, lo que permite medir la similitud de manera más efectiva.

   - *Eficiencia computacional*: Calcular la distancia del coseno es computacionalmente eficiente en comparación con otras métricas de similitud más complejas, como la distancia de Mahalanobis o la distancia euclidiana en espacios de alta dimensión. Esto es importante en sistemas de recomendación que manejan grandes conjuntos de datos y deben realizar cálculos rápidos para generar recomendaciones en tiempo real.

   - *Contexto de recomendación*: En muchos sistemas de recomendación, las preferencias de los usuarios pueden estar dispersas, lo que significa que solo han interactuado con una pequeña parte del conjunto total de elementos. La distancia del coseno tiende a funcionar bien en estos escenarios, ya que ignora las diferencias entre características no observadas, lo que puede conducir a una mejor generalización y recomendaciones más precisas.

A pesar de estas ventajas, es importante destacar que no existe un enfoque único para todos los sistemas de recomendación. La elección de la medida de similitud depende en gran medida del contexto y la naturaleza de los datos en un sistema específico. Por ejemplo,

 - En caso que las magnitudes sean importantes, la distancia euclidiana es una medida apropiada para utilizar. 
 - Si la correlación es lo que importa, entonces la correlación de Pearson o el coseno son más apropiadas. 
 - Si la popularidad del ítem bajo estudio es importante, entonces el producto escalar será una medida adecuada. Puesto que si los ítems aparecen con mucha frecuencia (por ejemplo, videos populares de YouTube) estos tienden a normas grandes y el producto escalar capturará mejor esta información. Sin embargo, si no somos cuidadosos, aquellos artículos populares terminarán siendo los más recomendados. En casos como este, se puede definir una medida que "regularice" las normas, como por ejemplo: $similitud(x,y)=|x|^\alpha  |y|^\alpha cos(x,y)$ para algún $\alpha\in(0,1)$.
 

##### Creamos una función recomendadora usando esto

In [6]:
# Importamos cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz de similitud en un DataFrame
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim


user_id,1,2,3,4,5
user_id,,,,,
1,1.000000,0.999630,0.997241,0.0,0.0
2,0.999630,1.000000,0.998891,0.0,0.0
3,0.997241,0.998891,1.000000,0.0,0.0
4,0.000000,0.000000,0.000000,1.0,1.0
5,0.000000,0.000000,0.000000,1.0,1.0


In [7]:
?cosine_similarity

Signature: cosine_similarity(X, Y=None, dense_output=True)
Docstring:
Compute cosine similarity between samples in X and Y.

Cosine similarity, or the cosine kernel, computes similarity as the
normalized dot product of X and Y:

.. code-block:: text

    K(X, Y) = <X, Y> / (||X||*||Y||)

On L2-normalized data, this function is equivalent to linear_kernel.

Read more in the :ref:`User Guide <cosine_similarity>`.

Parameters
----------
X : {array-like, sparse matrix} of shape (n_samples_X, n_features)
    Input data.

Y : {array-like, sparse matrix} of shape (n_samples_Y, n_features),             default=None
    Input data. If ``None``, the output will be the pairwise
    similarities between all samples in ``X``.

dense_output : bool, default=True
    Whether to return dense output even when the input is sparse. If
    ``False``, the output is sparse if both input arrays are sparse.

    .. versionadded:: 0.17
       parameter ``dense_output`` for dense output.

Returns
-------
similar

In [8]:
# Definimos una función para calcular el promedio ponderado de los ratings
def cf_user_wmean(user_id, restaurant_id):
    
    # Primero verificamos si el restaurante está en la matriz
    if restaurant_id in r_matrix:
    
        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]
        
        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[restaurant_id]
        
         # Obtenemos los índices de los restaurantes sin rating 
        idx = m_ratings[m_ratings.isnull()].index
        
        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()
        
        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0
    
    return wmean_rating

In [9]:
cf_user_wmean(1,1)

np.float64(4.000920501832994)

In [10]:
sim_scores = cosine_sim[1]
sim_scores

user_id
1    1.000000
2    0.999630
3    0.997241
4    0.000000
5    0.000000
Name: 1, dtype: float64

In [11]:
m_ratings =r_matrix[1]
m_ratings

user_id
1    5.0
2    4.0
3    3.0
4    NaN
5    NaN
Name: 1, dtype: float64

In [12]:
idx = m_ratings[m_ratings.isnull()].index
idx

Index([4, 5], dtype='int64', name='user_id')

Falta entonces 

$$
r_{um}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'm}}{\sum_{u',u'\neq u}sim(u,u')}
$$


In [13]:
sim_scores1 = sim_scores.drop(idx)

m_ratings = m_ratings.dropna()
        
# Calculamos la media ponderada
wmean_rating = np.dot(sim_scores1, m_ratings)/ sim_scores1.sum()
wmean_rating

np.float64(4.000920501832994)

## Un ejemplo más realista y complicado


Para ilustrar cómo construir e implementar este tipo de sistema de recomendación utilizaremos nuevamente los datos de  [MovieLens](https://grouplens.org/datasets/movielens/latest/) provista abiertamente por [grouplens](https://grouplens.org/about/what-is-grouplens/) para: **"avanzar la teoría y la práctica de la computación social mediante la construcción y la comprensión de sistemas *(de recomendación)* utilizados por personas reales".**

Esta versión de los datos contiene varias bases con información de los usuarios, de las películas, y de los ratings. Tendremos que combinar estas bases para obtener una matriz similar a la ilustrada en la gráfica anterior. Carguemos entonces las librerías y los datos de usuarios:

In [14]:
# Cargamos las librerías a utilizar
import pandas as pd
import numpy as np
# Cargamos los datos de los usuarios
u_cols = ['user_id', 'edad', 'genero', 'ocupacion', 'codigo_postal']

users = pd.read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/RecomSystemsLectures/main/L04_Usuarios/data/u.user', sep='|', names=u_cols,
 encoding='latin-1')
users.head()

,user_id,edad,genero,ocupacion,codigo_postal
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


Podemos ver que esta base contiene un identificador de usuario, su edad, género, ocupación, y el código postal donde viven. Luego cargamos la base de películas:

In [15]:
# Cargamos los datos de las películas
i_cols = ['movie_id','titulo', 'fecha_estreno', 'fecha_estreno_video', 'URL_IMDb', 'desconocido', 'Accion', 'Aventura',
  'Animacion', 'Infantil', 'Comedia', 'Crimen', 'Documental', 'Drama', 'Fantasia',
  'Cine-Noir', 'Horror', 'Musical', 'Misterio', 'Romance', 'Ciencia_ficcipn', 'Thriller', 'Guerra', 'Western']

movies = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/u.item', sep='|', names=i_cols, encoding='latin-1')

movies.head()

,movie_id,titulo,fecha_estreno,fecha_estreno_video,URL_IMDb,desconocido,Accion,Aventura,Animacion,Infantil,...,Fantasia,Cine-Noir,Horror,Musical,Misterio,Romance,Ciencia_ficcipn,Thriller,Guerra,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Esta base contiene un identificador de película, el título, cuándo fue estrenada en el cine, cuándo fue estrenada en video, la dirección web a IMDb, y el género al que pertenece, incluyendo una columna que marca como desconocido si el género no pertenece a ninguna de las restantes. Notemos además que una película puede pertenecer a varios géneros, por ejemplo: Toy Story pertenece  al genero Animación e Infantil. Pero por ahora nos importa sólo el identificador y el título de la película, por lo que nos quedamos con estas columnas:

In [16]:
## (Completar) Tip:  columnas de interes 'movie_id', 'titulo'
movies = movies[['movie_id', 'titulo']]
movies.head()

,movie_id,titulo
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


La última base que necesitamos cargar es la de ratings:

In [17]:
# Cargamos los datos de los ratings 
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/u.data', sep='\t', names=r_cols,
 encoding='latin-1')

ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Esta base tiene información sobre qué `rating` el usuario (`user_id`) le otorgó a cada película (`movie_id`), e información de cuándo fue que hizo tal calificación (`timestamp`). Esta última columna no la utilizaremos por lo que la excluiremos de la base.

In [18]:
# Quitamos la columna timestamp 
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


Con estos elementos podemos construir la matriz que vincule usuarios, películas y ratings. Para ello usamos la función `pivot_table`.

In [19]:
## (Completar) Tip: usar pivot_table siendo valores rating y las columnas movie_id
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

r_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
## (Completar) Tip:  la dimension con comando shape
r_matrix.shape


(943, 1682)

Esta matriz entonces vincula los usuarios con las películas. Las filas denotan los 943 usuarios y las columnas las 1682 películas en la base. El usuario 1 le otorgó a la película 1 un rating de 5, por lo tanto $r_{11}=5$. Al igual que en el ejemplo anterior, no todos los usuarios calificaron/vieron todas las películas y por lo tanto no todas tienen ranking. En el caso del usuario 1 este no calificó las películas 1673 a 1682 y por lo tanto aparecen con `NaN`; estos datos están faltando en la matriz.

La tarea es entonces buscar estrategias para completar estos datos faltantes.

### Medias

Comencemos por la estrategia quizás más simple e intuitiva para completar las celdas faltantes. Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario que ranquearon esta película. No hacemos distinción entre los usuarios, y el rating de cada uno recibirá el mismo peso. Habrá casos donde ninguna de las películas ha sido ranqueada, en tales situaciones le pondremos un rating default de 3. 

Para evaluar además la performance de esta estrategia, dividiremos la base original en bases de entrenamiento y prueba.

In [21]:
# Importamos la función train_test_split 
from sklearn.model_selection import train_test_split
   
#Asignamos `X` como la base original de ratings e `y` el usuario 
   
X = ratings.copy()
y = ratings['user_id']

# Partimos la base en entrenamiento y prueba estratificando por usuario
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

y definiremos como métrica el error cuadrático medio

In [22]:
# Importamos la función mean_squared_error
from sklearn.metrics import mean_squared_error

# Creamos una funcion que calcula la raíz del error cuadrático medio (RMSE)
def rmse(y_true, y_pred):
       return np.sqrt(mean_squared_error(y_true, y_pred))


Con la base lista podemos construir nuestra función recomendadora:

In [23]:
## Construir funcion de media

def cf_user_mean(user_id, movie_id):
    
    # Primero verificamos si la película está en la matriz
    if movie_id in r_matrix:
        # Si esta calculamos la media de los ratings
        mean_rating = r_matrix[movie_id].mean()
    
    else:
        # Si no lo está, asignamos el valor de 3 (los invito a que prueben con otros valores)
        mean_rating = 3.0
    
    return mean_rating

Podemos entonces evaluar el RMSE que obtendremos de la base de prueba de haber entrenado la función recomendadora en la base de entrenamiento:

In [24]:
def score(cf_model):
    # Construimos una lista con las tuplas usuario-película en la base de entrenamiento
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    # Predecimos el rating para cada tupla usuario-película
    y_pred = np.array([cf_model(user, movie) for (user, movie) in
   id_pairs])
    # Extraemos los ratings que dieron los usuarios en la base de prueba
    y_true = np.array(X_test['rating'])
    # Retornamos el RMSE
    return rmse(y_true, y_pred)

In [25]:
## (Completar) Tip: calcular el RMSE
score(cf_user_mean)

np.float64(1.0062161517166315)

Podemos entonces utlizar este resultado como una base para la comparación a los modelos subsiguientes.

### Medias ponderadas


In [26]:
# Rellenamos los faltantes con 0
r_matrix_dummy = r_matrix.copy().fillna(0)

In [27]:
r_matrix_dummy

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Con esto estamos en condiciones de usar la función `cosine_similarity` que va a calcular la similitud de coseno.

In [28]:
# Importamos cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz resultante en un dataframe
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941
6,0.430239,0.245843,0.072415,0.068044,0.237286,1.000000,0.489255,0.201369,0.183951,0.551713,...,0.385838,0.111828,0.190075,0.225142,0.137901,0.111852,0.352449,0.144446,0.317328,0.276042
7,0.440367,0.107328,0.066137,0.091230,0.373600,0.489255,1.000000,0.284951,0.145650,0.487024,...,0.456183,0.114179,0.112422,0.117860,0.153353,0.107027,0.329925,0.059993,0.282003,0.394364
8,0.319072,0.103344,0.083060,0.188060,0.248930,0.201369,0.284951,1.000000,0.085942,0.233289,...,0.239171,0.067626,0.094126,0.096483,0.169737,0.095898,0.246883,0.146145,0.175322,0.299809
9,0.078138,0.161048,0.061040,0.101284,0.056847,0.183951,0.145650,0.085942,1.000000,0.198223,...,0.082199,0.048640,0.163049,0.131415,0.118232,0.039852,0.120495,0.143245,0.092497,0.075617


Tenemos entonces como resultado una matriz que muestra la similitud de coseno entre los diferentes usuarios. La matriz diagonal que muestra $1$ nos dice que la similitud entre el usuario 1 y sí mismo es 1. Note que el 0.16 dice que la similitud entre 1 y 2 es de 0.16. En este caso, dado que estamos evaluando ratings positivos, esta matriz retornará entradas positivas.

Con esta matriz calculada podemos entonces utilizarla para construir nuestra función recomendadora basada en medias ponderadas.

In [29]:
## (Completar) Tip:  Crear función recomendadora basada en medias con input user_id y movie_id
def cf_user_wmean(user_id, movie_id):
    
    # Primero verificamos si la película esta en la matriz
    if movie_id in r_matrix:
    
        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]
        
        
        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[movie_id]
        
         # Obtenemos los índicies de las películas sin rating 
        idx = m_ratings[m_ratings.isnull()].index
        
        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()
        
        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0
    
    return wmean_rating

In [30]:
## (Completar) Tip: Probar la funcion
cosine_sim[1]

user_id
1      1.000000
2      0.166931
3      0.047460
4      0.064358
5      0.378475
         ...   
939    0.118095
940    0.314072
941    0.148617
942    0.179508
943    0.398175
Name: 1, Length: 943, dtype: float64

In [31]:
sim_scores = cosine_sim[1]
r_matrix[1]
m_ratings = r_matrix[1]
idx = m_ratings[m_ratings.isnull()].index
        
# Nos quedamos con similitudes y ratings completos
sim_scores = sim_scores.drop(idx)
m_ratings = m_ratings.dropna()
m_ratings

user_id
1      5.0
2      4.0
5      4.0
6      4.0
10     4.0
      ... 
934    2.0
935    3.0
936    4.0
938    4.0
941    5.0
Name: 1, Length: 452, dtype: float64

Estamos en condiciones entonces de evaluar este modelo:

In [32]:
## (Completar) Tip: evaluar.
score(cf_user_wmean)

np.float64(0.964396004447058)

Notemos que esta simple "sofisticación" mejora el desempeño predictivo.

### Datos demográficos de los usuarios

El modelo anterior entonces incorporó  similitud entre las calificaciones de los usuarios para mejorar las recomendaciones. ¿Qué sucede entonces si utilizamos variables demográficas? La intuición es que individuos de similares géneros, profesiones, edades, etc., tendrán preferencias más similares que individuos con características demográficas diferentes.

En este caso lo que haremos es restringir a los ratings de individuos con las mismas características demográficas, en jerga estadística, utilizaremos un modelo de medias condicionales.

Para ello, primero unimos los datos originales sobre usuarios con nuestra base de entrenamiento.

In [33]:
merged_df = pd.merge(X_train, users)

In [34]:
merged_df.head()

,user_id,movie_id,rating,edad,genero,ocupacion,codigo_postal
0,862,177,4,25,M,executive,13820
1,70,193,4,27,M,engineer,60067
2,666,527,4,44,M,administrator,61820
3,535,168,5,45,F,educator,80302
4,603,1240,5,21,M,programmer,47905


Con esta base que tiene ahora variables demográficas podemos intentar mejorar nuestro recomendador. Ilustremos entonces una función recomendadora basada en género y ocupación. Para ello, primero calculamos el rating promedio por género y ocupación. En `pandas` esto es relativamente sencillo utilizando la función `groupby`.

In [35]:
users = users.set_index('user_id')

In [36]:
# Calculamos el rating promedio basado en género y ocupación
# Completar
gen_occ_mean = merged_df[['genero', 'rating', 'movie_id', 'ocupacion']].pivot_table(
    values='rating', index='movie_id', columns=['ocupacion', 'genero'], aggfunc='mean')

gen_occ_mean.head()

ocupacion administrator           artist              doctor educator  \
genero                F         M      F         M         M        F   
movie_id                                                                
1              3.937500  3.750000    5.0  3.400000  3.666667     3.25   
2              3.000000  3.666667    NaN       NaN       NaN     4.00   
3              3.500000  4.000000    NaN       NaN       NaN      NaN   
4              3.666667  3.600000    NaN  4.666667  3.000000     2.50   
5              4.000000  2.333333    NaN       NaN       NaN     4.00   

ocupacion           engineer           entertainment  ... salesman       \
genero            M        F         M             F  ...        F    M   
movie_id                                              ...                 
1          3.884615      4.0  4.083333           4.0  ...      NaN  4.0   
2          3.500000      NaN  3.066667           NaN  ...      NaN  NaN   
3          2.000000      NaN  3.777778           NaN  ...      NaN  NaN   
4          3.800000      4.0  3.650000           NaN  ...      4.0  4.0   
5          2.333333      NaN  3.500000           NaN  ...      NaN  NaN   

ocupacion scientist        student           technician           writer  \
genero            F    M         F         M          F         M      F   
movie_id                                                                   
1               3.5  4.0  4.043478  3.796296        4.0  3.750000   4.00   
2               NaN  3.0  2.666667  3.277778        NaN  2.714286    NaN   
3               NaN  NaN  3.000000  3.391304        NaN  4.250000    NaN   
4               NaN  3.4  3.250000  3.777778        NaN  3.333333   4.25   
5               NaN  4.0  4.333333  3.111111        NaN  3.333333   4.00   

ocupacion            
genero            M  
movie_id             
1          3.000000  
2          2.333333  
3          1.000000  
4          3.250000  
5          2.000000  

[5 rows x 41 columns]

Tenemos entonces el promedio de rating por género y ocupación. Así, los administradores femeninos ranquearon la película 1 con un promedio de 3.93 y los masculinos con 3.75. Mientras que los artistas no ranquearon la película 2. Con esta información generamos una función recomendadora que incorpore esta información y en los casos donde no tenemos información utilizaremos el rating 3 como default.

In [37]:
## (Completar) Tip: generar las recomendaciones basadas en género y ocupación

def cf_gen_occ(user_id, movie_id):
    
    # Verificamos si la película existe en gen_occ_mean
    if movie_id in gen_occ_mean.index:
        
        #Identificamos el usuario
        user = users.loc[user_id]
        
        #Identificamos el género y la ocupación
        gender = user['genero']
        occ = user['ocupacion']
        
        # Verificamos si la ocupación calificó la película
        if occ in gen_occ_mean.loc[movie_id]:
            
            # Verificamos si el género calificó la película
            if gender in gen_occ_mean.loc[movie_id][occ]:
                
                # Obtenemos la calificación
                rating = gen_occ_mean.loc[movie_id][occ][gender]
                
                # Default de 3.0 si no tiene calificación
                if np.isnan(rating):
                    rating = 3.0
                
                return rating
            
    # Default de 3.0 si no tiene calificación
    return 3.0

Evaluamos entonces esta función recomendadora:

In [38]:
## (Completar) Tip: Evaluar el modelo de filtrado colaborativo basado en medias por género y ocupación
score(cf_gen_occ)

np.float64(1.1419651376788005)

Vemos que esta función es la que peor funciona de las antes evaluadas. La función que incorporaba el coseno como medida de similitud lograba mejores recomendaciones. En la sección siguiente nos centraremos en entender mejor las medidas de similitud y cómo estas nos permiten mejorar los recomendadores.

# Referencias

- Banik, R. (2018). Hands-on recommendation systems with Python: start building powerful and personalized, recommendation engines with Python. Packt Publishing Ltd.

- Covington, P., Adams, J., & Sargin, E. (2016). Deep Neural Networks for YouTube recommendations. Proceedings of the 10th ACM Conference on Recommender Systems. https://doi.org/10.1145/2959100.2959190 

- Google developers. (n.d.). Recommendation systems. Google. Consultado en Abril 3, 2022. Disponible en https://developers.google.com/machine-learning/recommendation/overview 

- Google developers. (n.d.). Embeddings: Motivation From Collaborative Filtering. Consultado en Mayo 13, 2022. Disponible en  https://developers.google.com/machine-learning/crash-course/embeddings/motivation-from-collaborative-filtering
